**Author: Miguel Félix**

Date: November 2024

###### Parameters definition for Calendar Start Date, Fiscal Year Date and Name of the table to be created in the lakehouse

In [1]:
start_date = "2001-01-01"
Fiscal_date = "2001-07-01"
LakehouseTableName = 'tblcalendardata'

StatementMeta(, f594ba6e-12a7-411f-b9d2-b7e8fba867e6, 3, Finished, Available, Finished)

###### SQL Code example for table generation (usage of parameter name of table)

In [2]:
df = spark.sql(f"""
DROP TABLE IF EXISTS {LakehouseTableName}; """)

StatementMeta(, f594ba6e-12a7-411f-b9d2-b7e8fba867e6, 4, Finished, Available, Finished)

###### Python code for generating table (usage of parameter name of table)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, lit, floor, weekofyear, datediff, current_date, dayofmonth, month, year, date_trunc, last_day

# Initialize Spark session
spark = SparkSession.builder.appName("CreateCalendarTable").getOrCreate()

# Define the start and end dates
# Start Data is the first fiscal date to be consider
# End date is based on current date
#start_date = "2012-01-01"
#Fiscal_date = "2012-07-01"
end_date = spark.sql("SELECT  concat(Year(current_date()) ,  '-12-31') AS current_date").collect()[0]['current_date']
# Add the number or years that you want to move the calendar forward (+ 2) and the end date that you need '-12-31'
#end_date = spark.sql("SELECT current_date() AS current_date").collect()[0]['current_date']

# Generate a DataFrame with all dates in the range
date_range_df = spark.sql(f"""
    WITH date_sequence AS (
        SELECT date_add('{start_date}', row_number() OVER (ORDER BY (SELECT NULL)) - 1) AS Date
        FROM range(datediff('{end_date}', '{start_date}') + 1)
    )
    SELECT Date FROM date_sequence
""")

# Define a function for calculating the end of the quarter
def calculate_quarter_end(date_col):
    return expr(f"last_day(add_months(date_trunc('quarter', {date_col}), 2))")

# Add additional columns to the DataFrame
calendar_df = date_range_df \
    .withColumn("Day", dayofmonth(col("Date")).cast("int")) \
    .withColumn("Day_name",expr("date_format(Date, 'EEEE')")) \
    .withColumn("Month", month(col("Date")).cast("int")) \
    .withColumn("Year", year(col("Date")).cast("int")) \
    .withColumn("Month_Name", expr("date_format(Date, 'MMMM')")) \
    .withColumn("Short_Month", expr("date_format(Date, 'MMM')")) \
    .withColumn("Quarter", floor((month(col("Date")) - 1) / 3) + 1) \
    .withColumn("Quarter_Name", expr("concat('Q', Quarter)")) \
    .withColumn("Quarter_Year", expr("concat(Quarter_Name,' ',substring(Year, -2))")) \
    .withColumn("Quarter_Year_YYYYQQ", expr("concat(Year, lpad(Quarter, 2, '0'))").cast("int")) \
    .withColumn("Month_Year_YYYYMM", expr("date_format(Date, 'yyyyMM')").cast("int")) \
    .withColumn("Month_Year_MMM_YY", expr("date_format(Date, 'MMM-yy')")) \
    .withColumn("Month_Year", expr("date_format(Date, 'MMM yy')")) \
    .withColumn("Is_Weekend", when(expr("date_format(Date, 'E')").isin("Sat", "Sun"), lit(1)).otherwise(lit(0)).cast("int")) \
    .withColumn("Week_Number", weekofyear(col("Date")).cast("int")) \
    .withColumn("First_Day_of_the_Week", date_trunc("week", col("Date")).cast("date")) \
    .withColumn("Last_Day_of_the_Week", date_trunc("week", col("Date")).cast("date") + 6) \
    .withColumn("Start_Date_of_the_Week_to_End_Date_of_the_Week", expr("(date_format((First_Day_of_the_Week), 'dd/MM/yyyy ')) ||'-'|| (date_format((Last_Day_of_the_Week), ' dd/MM/yyyy'))")) \
    .withColumn("First_Day_of_the_Month", date_trunc("month", col("Date")).cast("date")) \
    .withColumn("Day_of_Week", datediff(col("Date"), col("First_Day_of_the_Week")).cast("int") + 1) \
    .withColumn("Last_Day_of_the_Month", last_day(col("Date")).cast("date")) \
    .withColumn("First_Day_of_the_Quarter", date_trunc("quarter", col("Date")).cast("date")) \
    .withColumn("Last_Day_of_the_Quarter", calculate_quarter_end("Date").cast("date")) \
    .withColumn("Fiscal_Month_Name", expr("date_format(Date, 'MMMM')")) \
    .withColumn("Fiscal_Month_Number", when(month(col("Date")) >=  month(lit(Fiscal_date))   , month(col("Date")) - month(lit(Fiscal_date)) + 1 )
                                    .otherwise((12 - month(lit(Fiscal_date)))+ month(col("Date")) + 1).cast("int")) \
    .withColumn("Fiscal_Short_Month", expr("date_format(Date, 'MMM')")) \
    .withColumn("Fiscal_Quarter", when(col("Fiscal_Month_Number").between(1, 3), lit(1))
                                    .when(col("Fiscal_Month_Number").between(4, 6), lit(2))
                                    .when(col("Fiscal_Month_Number").between(7, 9), lit(3))
                                    .when(col("Fiscal_Month_Number").between(10, 12), lit(4)).cast("int")) \
    .withColumn("Fiscal_Quarter_Name", expr("concat('FY Q', Fiscal_Quarter)")) \
    .withColumn("Fiscal_Year", when(month(col("Date")) >= month(lit(Fiscal_date)), year(col("Date")) + 1).otherwise(year(col("Date"))).cast("int")) \
    .withColumn("Fiscal_Quarter_Year", expr("concat(Fiscal_Year, lpad(Fiscal_Quarter, 2, '0'))").cast("int")) \
    .withColumn("Fiscal_Quarter_Year_Name", expr("concat(Fiscal_Quarter_Name,' ', substring(Fiscal_Year, -2))")) \
    .withColumn("Fiscal_Year_Name", expr("concat('FY ', Fiscal_Year)")) \
    .withColumn("Fiscal_Week_Number", when(weekofyear(col("Date")) >= weekofyear(lit(Fiscal_date)), weekofyear(col("Date")) - weekofyear(lit(Fiscal_date)) + 1)
                                      .otherwise(53 - (weekofyear(lit(Fiscal_date)) - weekofyear(col("Date")))).cast("int")) \
    .withColumn("Fiscal_Month_Year_yyyyMM", expr("concat(Fiscal_Year, lpad(Fiscal_Month_Number, 2, '0'))").cast("int")) \
    .withColumn("Fiscal_Month_Year_MMM-YY", expr("concat(Fiscal_Short_Month, '-', substring(Fiscal_Year, -2))")) \
    .withColumn("Relative_Day", datediff(current_date(), col("Date")).cast("int")) \
    .withColumn("Relative_Fiscal_Month", (
        ((year(current_date()) - col("Fiscal_Year")) * 12) +
        (when(month(current_date()) >= 10, month(current_date()) - 9)
         .otherwise(3 + month(current_date())) - col("Fiscal_Month_Number"))
    ).cast("int"))
#Additional columns if needed
#.withColumn("Fiscal_First_Day_of_the_Week", date_trunc("week", col("Date")).cast("date")) \
#.withColumn("Fiscal_First_Day_of_the_Month", date_trunc("month", col("Date")).cast("date")) \
#.withColumn("Fiscal_Last_Day_of_the_Month", last_day(col("Date")).cast("date")) \
#.withColumn("Fiscal_First_Day_of_the_Quarter", date_trunc("quarter", col("Date")).cast("date")) \
#.withColumn("Fiscal_Last_Day_of_the_Quarter", calculate_quarter_end("Date").cast("date")) \


# Adjust the calculation to ensure the output is always positive
current_fiscal_year = year(current_date()) + (month(current_date()) >= month(lit(Fiscal_date))).cast("int")
current_fiscal_month = when(month(current_date()) >=  month(lit(Fiscal_date))  , month(current_date()) - month(lit(Fiscal_date)) +1   ).otherwise( (12 - month(lit(Fiscal_date))) + month(current_date()))
calendar_df = calendar_df.withColumn(
    "Relative_Fiscal_Month",
    ((current_fiscal_year - col("Fiscal_Year")) * 12 + (current_fiscal_month - col("Fiscal_Month_Number"))).cast("int")
)

# Write the DataFrame to the Lakehouse table

#calendar_df.show
calendar_df.write.mode("overwrite").saveAsTable(LakehouseTableName)


StatementMeta(, f594ba6e-12a7-411f-b9d2-b7e8fba867e6, 5, Finished, Available, Finished)

###### SQL Code to check data in table (hard coded table name)

In [4]:
%%sql
SELECT  * from tblcalendardata
order by Date desc
limit 1
;

StatementMeta(, f594ba6e-12a7-411f-b9d2-b7e8fba867e6, 6, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 38 fields>